In [1]:
! pip install mysql-connector-python

In [6]:
import mysql.connector 
import pandas as pd
def show(db, query):
    con = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        passwd = '12345Maskcu',
        database = db
    )
    
    executor = con.cursor()
    executor.execute(query)
    
    table = executor.fetchall()
    
    con.close()
    return pd.DataFrame(data=table, columns=executor.column_names)


In [7]:
query = 'SELECT * FROM actor LIMIT 20'

show(db='sakila', query=query)


,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33
5,6,BETTE,NICHOLSON,2006-02-15 04:34:33
6,7,GRACE,MOSTEL,2006-02-15 04:34:33
7,8,MATTHEW,JOHANSSON,2006-02-15 04:34:33
8,9,JOE,SWANK,2006-02-15 04:34:33
9,10,CHRISTIAN,GABLE,2006-02-15 04:34:33


#Q1 USING guitar_shop database

In [8]:
#1
q = '''
SELECT 
    DISTINCT(p1.product_name), 
    p1.list_price
FROM products p1, products p2
WHERE p1.list_price = p2.list_price 
    AND p1.product_id <> p2.product_id
ORDER BY p1.product_name
'''

show(db='guitar_shop', query=q)

,product_name,list_price
0,Fender Precision,799.99
1,Tama 5-Piece Drum Set with Cymbals,799.99


In [9]:
query = 'SELECT c.category_name, p.product_name, p.list_price FROM categories AS c INNER JOIN products AS p ON c.category_id = p.category_id ORDER BY c.category_name, p.product_name ASC'
show (db = 'guitar_shop', query=query)

,category_name,product_name,list_price
0,Basses,Fender Precision,799.99
1,Basses,Hofner Icon,499.99
2,Drums,Ludwig 5-piece Drum Set with Cymbals,699.99
3,Drums,Tama 5-Piece Drum Set with Cymbals,799.99
4,Guitars,Fender Stratocaster,699.00
5,Guitars,Gibson Les Paul,1199.00
6,Guitars,Gibson SG,2517.00
7,Guitars,Rodriguez Caballero 11,415.00
8,Guitars,Washburn D10S,299.00
9,Guitars,Yamaha FG700S,489.99


1. Write a `SELECT` statement that returns the product_name and list_price columns from the Products table.
    - Return one row for each product that has the same list price as another product.
    - Sort the result set by product_name.

In [7]:
q = '''SELECT 
    DISTINCT(p1.product_name), 
    p1.list_price
FROM products p1, products p2
WHERE p1.list_price = p2.list_price 
    AND p1.product_id <> p2.product_id
ORDER BY p1.product_name'''
show (db='guitar_shop', query=q)

,product_name,list_price
0,Fender Precision,799.99
1,Tama 5-Piece Drum Set with Cymbals,799.99


2. Write a `SELECT` statement that returns these two columns:
    - category_name
    - product_id
    Return one row for each category that has never been used.

In [11]:
q = '''
SELECT category_name, product_id
FROM categories LEFT JOIN products USING (category_id)
WHERE category_id is NULL
'''

show(db='guitar_shop', query=q)

,category_name,product_id


3. Write a `SELECT` statement that returns one row for each category that has products with these columns:
    - The category_name column from the Categories table
    - The count of the products in the Products table
    - The list price of the most expensive product in the Products table
    Sort the result set so the category with the most products appears first.

In [12]:
q = '''
SELECT 
    category_name, 
    COUNT(product_id) AS product_count, 
    MAX(list_price) AS most_expensive_product
FROM categories JOIN products USING (category_id)
GROUP BY category_name
ORDER BY product_count DESC
'''

show(db='guitar_shop', query=q)

,category_name,product_count,most_expensive_product
0,Guitars,6,2517.00
1,Basses,2,799.99
2,Drums,2,799.99


4. Write a `SELECT` statement that returns one row for each customer that has orders with these columns:
    - The email_address from the Customers table
    - A count of the number of orders
    - The total amount for each order
    
    Return only those rows where the customer has more than 1 order.
    Sort the result set in descending sequence by the sum of the line item amounts.

In [13]:
q = '''
SELECT 
    email_address,
    COUNT(order_id) AS order_count, 
    SUM((item_price - discount_amount) * quantity) AS order_total
FROM customers
    JOIN orders USING (customer_id)
    JOIN order_items USING (order_id)
GROUP BY email_address
HAVING COUNT(*) > 1
ORDER BY order_total DESC
'''

show(db='guitar_shop', query=q)

,email_address,order_count,order_total
0,allan.sherwood@yahoo.com,3,2300.61
1,frankwilson@sbcglobal.net,3,1539.28
2,david.goldstein@hotmail.com,2,788.30


#Q2 USING saklia database

1. Write a `SELECT` statement that returns these columns:
    - First and last names of staff members
    - Address

In [14]:
# query = ''' 
# SELECT
# concat(first_name, ' ' last_name) as fullname,
# address FROM customer c JOIN address a ON c.address_id = a.address_id limit 20
# '''
# show (db='sakila', query=query)

q = '''
SELECT 
    CONCAT(first_name, ' ',last_name) AS fullname, 
    address
FROM staff JOIN address USING (address_id)
'''

show(db='sakila', query=q)

,fullname,address
0,Mike Hillyer,23 Workhaven Lane
1,Jon Stephens,1411 Lillydale Drive


2. Write a `SELECT` statement that returns these columns:
    - Staff Id
    - First and last names
    - Total amount
    Show total amount rung up by each staff member in August of 2005.

In [16]:
q = '''
SELECT 
    staff_id,
    CONCAT(first_name, ' ', last_name) AS employee_name,
    CONCAT('$', FORMAT(SUM(amount), 2)) AS amount
FROM staff JOIN payment USING (staff_id)
WHERE YEAR(payment_date) = 2005 AND MONTH(payment_date) = 8
GROUP BY staff_id
'''

show(db='sakila', query=q)

,staff_id,employee_name,amount
0,1,Mike Hillyer,"$11,853.65"
1,2,Jon Stephens,"$12,218.48"


3. Write a `SELECT` statement that returns these columns:
    - Title
    - Number of rentals
    
    Show the most frequently rented movies in descending order.

In [17]:
q = '''
SELECT 
    title, COUNT(rental_id) AS 'Number of Rentals'
FROM film
    JOIN inventory USING (film_id)
    JOIN rental USING (inventory_id)
GROUP BY title
ORDER BY 2 DESC
'''

a = show(db='sakila', query=q)
a

,title,Number of Rentals
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,FORWARD TEMPLE,32
3,GRIT CLOCKWORK,32
4,JUGGLER HARDLY,32
...,...,...
953,SEVEN SWARM,5
954,TRAFFIC HOBBIT,5
955,HARDLY ROBBERS,4
956,MIXED DOORS,4


4. You want to run an email marketing campaign in Indonesia, for which you will need the names and email addresses of all Indonesia customers.

In [18]:
q = '''
SELECT 
    CONCAT(first_name, ' ', last_name) AS fullname,
    email
FROM customer
        JOIN address USING (address_id)
        JOIN city USING (city_id)
        JOIN country USING (country_id)
WHERE
    country = 'Indonesia'
'''

show(db='sakila', query=q)

,fullname,email
0,NORMAN CURRIER,NORMAN.CURRIER@sakilacustomer.org
1,MINNIE ROMERO,MINNIE.ROMERO@sakilacustomer.org
2,JEFFREY SPEAR,JEFFREY.SPEAR@sakilacustomer.org
3,TARA RYAN,TARA.RYAN@sakilacustomer.org
4,SUZANNE NICHOLS,SUZANNE.NICHOLS@sakilacustomer.org
5,JORGE OLIVARES,JORGE.OLIVARES@sakilacustomer.org
6,REGINALD KINDER,REGINALD.KINDER@sakilacustomer.org
7,LONNIE TIRADO,LONNIE.TIRADO@sakilacustomer.org
8,VICTORIA GIBSON,VICTORIA.GIBSON@sakilacustomer.org
9,LESLIE SEWARD,LESLIE.SEWARD@sakilacustomer.org


#Q3 Use employees

1. Write a `SELECT` statement that returns these columns:
    - First and last name
    - Number of departments
    
    Find all the employees that have worked in at least 2 departments. And display all results in ascending order.

In [19]:
q = '''
SELECT 
    CONCAT(first_name, ' ', last_name) AS fullname,
    COUNT(*) AS number_of_departments
FROM employees JOIN dept_emp USING (emp_no)
GROUP BY emp_no
HAVING COUNT(*) > 1
ORDER BY 1
'''

show(db='employees', query=q)

,fullname,number_of_departments
0,Aamer Baba,2
1,Aamer Cyre,2
2,Aamer Danner,2
3,Aamer Frolund,2
4,Aamer Koprowski,2
...,...,...
31574,Zvonko Sabnani,2
31575,Zvonko Schauser,2
31576,Zvonko Schwaller,2
31577,Zvonko Serot,2


2. Write a `SELECT` statement that returns these columns:
    - First name of the employee
    - First name of their manager.
    
    Find the employees and their managers. Those managers do not work under any manager also appear in the list.

In [20]:
q = '''
SELECT *
FROM dept_manager
LIMIT 5
'''

show(db='employees', query=q)

,emp_no,dept_no,from_date,to_date
0,110022,d001,1985-01-01,1991-10-01
1,110039,d001,1991-10-01,9999-01-01
2,110085,d002,1985-01-01,1989-12-17
3,110114,d002,1989-12-17,9999-01-01
4,110183,d003,1985-01-01,1992-03-21


In [26]:
q = '''
SELECT *
FROM dept_emp
LIMIT 5
'''

show(db='employees', query=q)

,emp_no,dept_no,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01


In [27]:
q = '''
SELECT *
FROM employees
LIMIT 5
'''

show(db='employees', query=q)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [28]:
q = '''
SELECT 
    DISTINCT(CONCAT(e2.first_name, ' ', e2.last_name)) AS employe_name, 
    CONCAT(e1.first_name, ' ', e1.last_name) AS manager_name
FROM dept_manager dm
		JOIN dept_emp de ON dm.dept_no = de.dept_no
		JOIN employees e1 ON dm.emp_no = e1.emp_no
		JOIN employees e2 ON de.emp_no = e2.emp_no
'''

show(db='employees', query=q)

,employe_name,manager_name
0,Aamer Anger,Marjo Giarratana
1,Aamer Anger,Tonny Butterworth
2,Aamer Anger,Xiaobin Spinelli
3,Aamer Anger,Yuchang Weedman
4,Aamer Armand,Dung Pesch
...,...,...
886021,Zvonko Zuberek,Krassimir Wegerle
886022,Zvonko Zuberek,Leon DasSarma
886023,Zvonko Zuberek,Oscar Ghazalie
886024,Zvonko Zuberek,Rosine Cools


3. Write a `SELECT` statement that returns these columns:
    - Name of department
    - Age in year
    
    Display each department and the age of the youngest employee at hire date.

In [29]:
q = '''
SELECT 
    dept_name, 
    MIN(TIMESTAMPDIFF(YEAR, birth_date, hire_date)) AS employee_age
FROM employees 
	JOIN dept_emp USING (emp_no)
	JOIN departments USING (dept_no)
GROUP BY dept_name
'''

show(db='employees', query=q)

,dept_name,employee_age
0,Customer Service,20
1,Development,20
2,Finance,20
3,Human Resources,20
4,Marketing,20
5,Production,20
6,Quality Management,20
7,Research,20
8,Sales,20


4. Write a `SELECT` statement that returns these columns:
    - Employee number
    - First and last name
    
    Find employees whose average earnings are less than the average salary in that company.

In [30]:
q = '''
SELECT 
    emp_no, 
    CONCAT(first_name, ' ', last_name) AS fullname,
    AVG(salary) AS rerata
FROM employees JOIN salaries USING (emp_no)
GROUP BY emp_no
HAVING rerata < (SELECT AVG(salary) FROM salaries)
'''

show(db='employees', query=q)

,emp_no,fullname,rerata
0,10003,Parto Bamford,43030.2857
1,10004,Chirstian Koblick,56512.2500
2,10006,Anneke Preusig,50514.9167
3,10008,Saniya Kalloufi,49307.6667
4,10011,Mary Sluis,49782.0000
...,...,...,...
189172,499993,DeForest Mullainathan,40944.6667
189173,499994,Navin Argence,40000.0000
189174,499995,Dekang Lichtner,45891.6667
189175,499996,Zito Baaz,63134.1429


#Use world database

1. Get the number of cities in the IDN.

In [31]:
q = '''
SELECT COUNT(DISTINCT(Name)) AS n_cities
FROM city
WHERE countrycode = 'IDN'
'''

show(db='world', query=q)

,n_cities
0,84


2. What is the head of state of Indonesia (IDN)?

In [32]:
q = '''
SELECT headofstate AS 'Presiden'
FROM country
WHERE code = 'IDN'
'''

show(db='world', query=q)

,Presiden
0,Abdurrahman Wahid


3. What is the capital of Indonesia (IDN)?

In [33]:
q = '''
SELECT city.name, country.name
FROM country 
    JOIN city ON country.code = city.CountryCode
WHERE city.CountryCode = 'IDN' AND city.ID = country.Capital
'''

show(db='world', query=q)

,name,name
0,Jakarta,Indonesia


4. List all the languages spoken in the 'Southeast Asia' region.

In [34]:
q = '''
SELECT cl.language
FROM countrylanguage cl LEFT JOIN country c 
    ON cl.CountryCode = c.code
WHERE c.region = 'Southeast Asia'
'''

show(db='world', query=q)

,language
0,Chinese
1,English
2,Malay
3,Malay-English
4,Bali
...,...
60,Muong
61,Nung
62,Thai
63,Tho
